In [1]:
import pandas as pd
from osgeo import gdal
import math
# from PIL import Image # PIL library supports only TIF format but not newer TIFF

In [81]:
data_directory = '../data/'
asf_data_directory = data_directory + '3_processed_ASF_data/'
interpolated_AIS_data_directory = data_directory + '5_interpolated_AIS_data/'
cutted_asf_data_directory = data_directory + '6_cutted_ASF_data/'
annotated_asf_data_directory = data_directory + '6_annotated_ASF_data/'
sliced_asf_data_directory = data_directory +'7_sliced_ASF_data/'
results_directory = '../results/'

In [61]:
SHIP_WIDTH_PX = 50
SHIP_HEIGHT_PX = 50
OBJECT_CLASS = 0

In [4]:
full_data_df = pd.read_csv(results_directory + "2022-12-15_03-46-06_full_data_df.csv", index_col=0)
full_data_df

,asf_file,image_datetime,date,minLON,minLAT,maxLON,maxLAT,widthpx,heightpx,datetime_lower,datetime_upper,ais_file
0,S1A_IW_GRDH_1SDV_20210204T015812_20210204T0158...,2021-02-04 01:58:26.500,2021-02-04,-118.496252,33.197789,-117.964359,33.855266,5921,7319,2021-02-04 01:48:26.500,2021-02-04 02:08:26.500,AIS_2021_02_04.csv
1,S1B_IW_GRDH_1SDV_20210105T015737_20210105T0158...,2021-01-05 01:57:51.500,2021-01-05,-118.575269,33.251929,-117.946359,33.930247,7001,7551,2021-01-05 01:47:51.500,2021-01-05 02:07:51.500,AIS_2021_01_05.csv
2,S1B_IW_GRDH_1SDV_20210306T015735_20210306T0158...,2021-03-06 01:57:49.500,2021-03-06,-118.482835,33.199482,-117.980677,33.829740,5590,7016,2021-03-06 01:47:49.500,2021-03-06 02:07:49.500,AIS_2021_03_06.csv


# Huge Image ships labelling
https://stackoverflow.com/questions/8898120/conversion-of-lat-lng-coordinates-to-pixels-on-a-given-map-with-javascript

    Formula - Longitude - pixel = (givenLng * widthOfContainerElement) / 360

    where 360 is the total longitude in degrees

    Formula -Latitude - pixed = (givenLat * heightOfContainerElement) / 180

    def convert_geographic_coordinate_to_pixel_value(lon, lat, transform):
        """
        Converts a latitude/longitude coordinate to a pixel coordinate given the
        geotransform of the image.
        Args:
            lon: Pixel longitude.
            lat: Pixel latitude.
            transform: The geotransform array of the image.
        Returns:
            Tuple of refx, refy pixel coordinates.
        """

        xOrigin = transform[0]
        yOrigin = transform[3]
        pixelWidth = transform[1]
        pixelHeight = -transform[5]

        refx = round((lon - xOrigin) / pixelWidth)
        refy = round((yOrigin - lat) / pixelHeight)

        return refx, refy

In [25]:
def calculate_px_from_lon(row, image_width):  
    return row['LON']*image_width / 360

In [26]:
def calculate_px_from_lat(row, image_height):  
    return row['LAT']*image_height / 180

# Methods down are more proper

In [27]:
def calculate_px_from_lon2(row, xOrigin, pixelWidth):  
    return round((row['LON'] - xOrigin) / pixelWidth)

In [28]:
def calculate_px_from_lat2(row, yOrigin, pixelHeight):  
    return round((yOrigin - row['LAT']) / pixelHeight)

Image YOLO format normalization: 

    Darknet label format: [label_index, xcenter, ycenter, w, h] (Relative coordinates)
    For example:
    xmin: top-left x coordinate of the box,
    ymin: top-left y coordinate of the box,
    w: bounding box width,
    h: bounding box height,
    w_img: image width,
    h_img: image height

    xcenter = (xmin + w/2) / w_img
    ycenter = (ymin + h/2) / h_img
    w = w / w_img
    h = h / h_img

In [54]:
full_data_df

,asf_file,image_datetime,date,minLON,minLAT,maxLON,maxLAT,widthpx,heightpx,datetime_lower,datetime_upper,ais_file
0,S1A_IW_GRDH_1SDV_20210204T015812_20210204T0158...,2021-02-04 01:58:26.500,2021-02-04,-118.496252,33.197789,-117.964359,33.855266,5921,7319,2021-02-04 01:48:26.500,2021-02-04 02:08:26.500,AIS_2021_02_04.csv
1,S1B_IW_GRDH_1SDV_20210105T015737_20210105T0158...,2021-01-05 01:57:51.500,2021-01-05,-118.575269,33.251929,-117.946359,33.930247,7001,7551,2021-01-05 01:47:51.500,2021-01-05 02:07:51.500,AIS_2021_01_05.csv
2,S1B_IW_GRDH_1SDV_20210306T015735_20210306T0158...,2021-03-06 01:57:49.500,2021-03-06,-118.482835,33.199482,-117.980677,33.829740,5590,7016,2021-03-06 01:47:49.500,2021-03-06 02:07:49.500,AIS_2021_03_06.csv


In [80]:
for index, row in full_data_df.iterrows():
    print(f"File: {index}")
    asf_file_name = row['asf_file']
    print(f"ASF file name: {file_name}")
    ais_file_name = 'interpolated_processed_' + row['ais_file']
    print(f"AIS file name: {ais_file_name}")
    ais_file_df = pd.read_csv(interpolated_AIS_data_directory + ais_file_name, index_col=0)
    display(ais_file_df.head())
    in_ds = gdal.Open(asf_data_directory + file_name)
    #ds = gdal.Open(asf_data_directory + 'S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.tif')
    image_width = in_ds.RasterXSize
    image_height = in_ds.RasterYSize
    geo_transform = in_ds.GetGeoTransform()
    minx = geo_transform[0]
    miny = geo_transform[3] + image_width*geo_transform[4] + image_height*geo_transform[5] 
    xOrigin = geo_transform[0]
    yOrigin = geo_transform[3]
    pixelWidth = geo_transform[1]
    pixelHeight = -geo_transform[5]
    print(f"image_width: {image_width}, image_height: {image_height}")
    print(f"pixel_width: {pixelWidth}, pixel_height: {pixelHeight}")
    print(f"xOrigin: {xOrigin}, yOrigin: {yOrigin}")
    # image_height = row['heightpx']
    # image_width = row['widthpx']
    print(f"Image_size: {image_width}x{image_height} px")
    ais_file_df['LONpx_X'] = ais_file_df.apply(lambda row: calculate_px_from_lon(row, image_width), axis=1)
    ais_file_df['LATpx_Y'] = ais_file_df.apply(lambda row: calculate_px_from_lat(row, image_height), axis=1)
    ais_file_df['LONpx_X2'] = ais_file_df.apply(lambda row: calculate_px_from_lon2(row, xOrigin, pixelWidth), axis=1)
    ais_file_df['LATpx_Y2'] = ais_file_df.apply(lambda row: calculate_px_from_lat2(row, yOrigin, pixelHeight), axis=1)
    display(ais_file_df.head())
    
    file_object = open(annotated_asf_data_directory + asf_file_name.strip('.tif') + '.txt', 'a')
    for inx, rw in ais_file_df.iterrows():
        #OBJECT_CLASS, SHIP_WIDTH_PX, SHIP_HEIGHT_PX
        #print("before normalization")
        X_CENTER_AXIS_VALUE = rw['LONpx_X2']
        Y_CENTER_AXIS_VALUE = rw['LATpx_Y2']
        #print(OBJECT_CLASS, X_CENTER_AXIS_VALUE, Y_CENTER_AXIS_VALUE, SHIP_WIDTH_PX, SHIP_HEIGHT_PX)
        #print("after normalization")
        #X_CENTER_AXIS_VALUE = (X_CENTER_AXIS_VALUE + SHIP_WIDTH_PX/2) / image_width
        #Y_CENTER_AXIS_VALUE = (Y_CENTER_AXIS_VALUE + SHIP_HEIGHT_PX/2) / image_height
        X_CENTER_AXIS_VALUE = X_CENTER_AXIS_VALUE / image_width
        Y_CENTER_AXIS_VALUE = Y_CENTER_AXIS_VALUE / image_height
        SHIP_WIDTH_normalized = SHIP_WIDTH_PX / image_width
        SHIP_HEIGHT_normalized = SHIP_HEIGHT_PX / image_height
        #print(OBJECT_CLASS, X_CENTER_AXIS_VALUE, Y_CENTER_AXIS_VALUE, SHIP_WIDTH_normalized, SHIP_HEIGHT_normalized)
        #break
        string_to_write = f"{OBJECT_CLASS} {X_CENTER_AXIS_VALUE} {Y_CENTER_AXIS_VALUE} {SHIP_WIDTH_normalized} {SHIP_HEIGHT_normalized}\n"
        file_object.write(string_to_write)
        # Close the file
        
    file_object.close()
    break

File: 0
ASF file name: S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.tif
AIS file name: interpolated_processed_AIS_2021_02_04.csv


,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
0,338060204,2021-02-04 01:58:26.500,33.720990,-118.279090,0.0,360.0,511.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,A
1,636092268,2021-02-04 01:58:26.500,33.641630,-118.133710,0.0,184.2,262.0,MSC RAVENNA,IMO9484431,A8ZU8,70.0,1.0,365.0,51.0,16.0,NaN,A
2,338162425,2021-02-04 01:58:26.500,33.722810,-118.068750,0.0,271.0,511.0,CHELSEA SKYE TWO,IMO0000000,NaN,37.0,NaN,0.0,0.0,NaN,NaN,B
3,338229302,2021-02-04 01:58:26.500,33.753294,-118.111120,0.0,297.3,511.0,NAUTICAL SMILE,NaN,NaN,37.0,NaN,NaN,NaN,NaN,NaN,B
4,338100018,2021-02-04 01:58:26.500,33.848660,-118.399311,0.0,292.0,511.0,MARE'ZIA,NaN,NaN,37.0,NaN,12.0,3.0,NaN,NaN,B


image_width: 5921, image_height: 7319
pixel_width: 8.983152841195215e-05, pixel_height: 8.983152841195215e-05
xOrigin: -118.4962518528152, yOrigin: 33.855265740629456
Image_size: 5921x7319 px


,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,...,Status,Length,Width,Draft,Cargo,TranscieverClass,LONpx_X,LATpx_Y,LONpx_X2,LATpx_Y2
0,338060204,2021-02-04 01:58:26.500,33.720990,-118.279090,0.0,360.0,511.0,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,A,-1945.362477,1371.132930,2417,1495
1,636092268,2021-02-04 01:58:26.500,33.641630,-118.133710,0.0,184.2,262.0,MSC RAVENNA,IMO9484431,A8ZU8,...,1.0,365.0,51.0,16.0,NaN,A,-1942.971373,1367.906037,4036,2378
2,338162425,2021-02-04 01:58:26.500,33.722810,-118.068750,0.0,271.0,511.0,CHELSEA SKYE TWO,IMO0000000,NaN,...,NaN,0.0,0.0,NaN,NaN,B,-1941.902969,1371.206915,4759,1474
3,338229302,2021-02-04 01:58:26.500,33.753294,-118.111120,0.0,297.3,511.0,NAUTICAL SMILE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,B,-1942.599838,1372.446451,4287,1135
4,338100018,2021-02-04 01:58:26.500,33.848660,-118.399311,0.0,292.0,511.0,MARE'ZIA,NaN,NaN,...,NaN,12.0,3.0,NaN,NaN,B,-1947.339785,1376.324134,1079,74


In [37]:
ais_file_df.LONpx_X.min(), ais_file_df.LONpx_X.max()

(-1948.933978117294, -1939.9240597164655)

In [38]:
ais_file_df.LATpx_Y.min(), ais_file_df.LATpx_Y.max()

(1353.9935851481482, 1376.4027453164251)

In [35]:
ais_file_df.LONpx_X2.min(), ais_file_df.LONpx_X2.max()

(0, 6098)

In [36]:
ais_file_df.LATpx_Y2.min(), ais_file_df.LATpx_Y2.max()

(52, 6187)

# Image slicing

In [78]:
from sahi.slicing import slice_image

In [82]:
slice_image_result, num_total_invalid_segmentation = slice_image(
    image=annotated_asf_data_directory + 'S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.tif',
    output_file_name='S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC',
    output_dir=sliced_asf_data_directory,
    slice_height=256,
    slice_width=256,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
)

IndexError: tuple index out of range